In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import os
from scipy.stats import norm



In [ ]:
data_path = os.path.join("..", "data")

In [ ]:
X = np.load(data_path + '/initial_data/function_5/initial_inputs.npy')
y = np.load(data_path + '/initial_data/function_5/initial_outputs.npy')
print(f"X is:{X}")
print(f"Y is :{y}")

In [ ]:
# add the week 1 input and  result to the array
X = np.append(X, [[0.224489, 0.836734, 0.877551, 0.877551]], axis=0)
y = np.append(y, [1035.7833689308636], axis=0)

# add the week 2 input and result to the array
X = np.append(X, [[0.204082, 0.877551, 0.857143, 0.897959]], axis=0)
y = np.append(y, [1221.5705853873078], axis=0)

# add the week 3 input and result to the array
X = np.append(X, [[0.220562, 0.993718, 0.923250, 0.932260]], axis=0)
y = np.append(y, [2763.136135055027], axis=0)

# add the week 4 input and result to the array
X = np.append(X, [[0.210582, 0.914111, 0.854212, 0.943472]], axis=0)
y = np.append(y, [1699.4414716139875], axis=0)

# add the week 5 input and result to the array
X = np.append(X, [[0.223249, 0.991019, 0.987207, 0.989510]], axis=0)
y = np.append(y, [4021.5011695428084], axis=0)

# add the week 6 input and result to the array
X = np.append(X, [[0.210130, 0.932370, 0.868367, 0.947575]], axis=0)
y = np.append(y, [1938.9432672210037], axis=0)

# add the week 7 input and result to the array
X = np.append(X, [[0.256514, 0.997140, 0.999654, 0.992373]], axis=0)
y = np.append(y, [4310.980559441547], axis=0)

# add the week 8 input and result to the array
X = np.append(X, [[0.331337, 0.998687, 0.980926, 0.995698]], axis=0)
y = np.append(y, [4158.78669464844], axis=0)

# add the week 9 input and result to the array
X = np.append(X, [[0.260000, 0.995000, 0.995000, 0.990000]], axis=0)
y = np.append(y, [4189.1605631845205], axis=0)

# add the week 10 input and result to the array
X = np.append(X, [[0.565589, 0.990737, 0.998529, 0.990139]], axis=0)
y = np.append(y, [4537.8399923503885], axis=0)

# add the week 11 input and result to the array
X = np.append(X, [[0.405247, 0.994863, 0.998796, 0.997793]], axis=0)
y = np.append(y, [4427.020121180915], axis=0)

# add the week 12 input and result to the array
X = np.append(X, [[0.693927, 0.995985, 0.998781, 0.984237]], axis=0)
y = np.append(y, [5012.317575448201], axis=0)

print(f"X is:{X}")
print(f"Y is :{y}")

In [ ]:
print(f"max y is :{np.max(y)}")

In [ ]:
# Define GP model
kernel = Matern(nu=2.5, length_scale_bounds=(1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, normalize_y=True)
gp.fit(X, y)

In [ ]:
# UCB acquisition function
# week 6 changed kappa to 4.0 from 2.0
# week 7 changed kappa to 1.0 from 4.0 for exploitation
# week 8 changed kappa to 0.5 for more exploitation
# week 10 changed kappa to 1 for more exploitation
# week 11 kept  kappa to 1 for more exploitation
def ucb(x, gp, kappa=1):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return mu + kappa * sigma

In [ ]:
# Expected Improvement (EI) acquisition function 
def expected_improvement(x, gp, y_best, xi=0.01):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    
    if sigma == 0:
        return 0.0
    
    Z = (mu - y_best - xi) / sigma
    ei = (mu - y_best - xi) * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

In [ ]:
"""grid_size = 50
x1 = np.linspace(0, 1, grid_size)
x2 = np.linspace(0, 1, grid_size)
x3 = np.linspace(0, 1, grid_size)
x4 = np.linspace(0, 1, grid_size)
X_grid = np.array([[a, b, c, d] for a in x1 for b in x2 for c in x3 for d in x4])"""

In [ ]:
n_samples = 3000000 # You can adjust this based on your compute budget

# Generate random 6D points in [0, 1]^6
X_random = np.random.rand(n_samples, 4)

In [ ]:
ucb_values = np.array([ucb(x, gp) for x in X_random])


In [ ]:
y_best = np.max(y)
ei_values = np.array([expected_improvement(x, gp, y_best) for x in X_random])

In [ ]:
best_idx = np.argmax(ucb_values)
next_best_point = X_random[best_idx]
print("Next best point to evaluate (UCB) rounded:", np.round(next_best_point,6))
print("Next best point to evaluate (UCB):", next_best_point)
pred_mean, pred_std = gp.predict(next_best_point.reshape(1, -1), return_std=True)
print("Predicted maximum value (mean):", pred_mean[0])

In [ ]:
# Select next best point
best_idx = np.argmax(ei_values)
next_best_point = X_random[best_idx]
pred_mean, pred_std = gp.predict(next_best_point.reshape(1, -1), return_std=True)

print("Next best point to evaluate (EI):", np.round(next_best_point, 6))
print("Next best point to evaluate (EI) without rounding:", next_best_point)
print("Predicted mean at next point:", pred_mean[0])
print("Expected Improvement value:", ei_values[best_idx])

In [ ]:
# SVM model as surrogate
# Scale inputs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train SVM surrogate
svm_surrogate = SVR(kernel='rbf', C=5000.0, epsilon=2.0)
svm_surrogate.fit(X_scaled, y)

# Predict on new candidates
X_new = np.random.rand(10000000, 4)
X_new_scaled = scaler.transform(X_new)
y_pred = svm_surrogate.predict(X_new_scaled)

# Maximization: find best candidate
best_idx = np.argmax(y_pred)
best_candidate = X_new[best_idx]
best_predicted_value = y_pred[best_idx]

print("Best candidate input:", best_candidate)
print("Best candidate input rounded:", np.round(best_candidate, 6))
print("Predicted maximum value:", best_predicted_value)